In [ ]:
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from textwrap import dedent
from dotenv import load_dotenv

load_dotenv()

# 벡터 저장 디렉토리
WEBTOON_DB_PATH = "../../../data/Raw_DB/vector_store/rofan_webtoon"
WEBNOVEL_DB_PATH = "../../../data/Raw_DB/vector_store/rofan_webnovel"
TOTAL_DB_PATH =  "../../../data/Raw_DB/vector_store/rofan_total"

# ChromaDB 연결 (웹툰 & 웹소설 & 전체)
embedding_model = HuggingFaceEmbeddings(model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr")

webtoon_db = Chroma(
    collection_name="rofan_webtoon",
    persist_directory=WEBTOON_DB_PATH,
    embedding_function=embedding_model
)

webnovel_db = Chroma(
    collection_name="rofan_webnovel",
    persist_directory=WEBNOVEL_DB_PATH,
    embedding_function=embedding_model
)

total_db = Chroma(
    collection_name="rofan_total",
    persist_directory=TOTAL_DB_PATH,
    embedding_function=embedding_model
)

# 리트리버 생성
webtoon_retriever = webtoon_db.as_retriever(search_type="similarity", search_kwargs={"k": 5})
webnovel_retriever = webnovel_db.as_retriever(search_type="similarity", search_kwargs={"k": 5})
total_retriever = total_db.as_retriever(search_type="similarity", search_kwargs={"k": 5})


{'제목': '도망친 곳이 낙원이었다 [19세 완전판]', '타입': '웹소설', '플랫폼': '카카오페이지', '장르': '로판', '키워드': ['로맨스판타지', '회귀물', '직진남', '능력남', '도도녀', '까칠녀', '달달물', '성장물'], '줄거리': "7번의 결혼, 5번의 사별, 1번의 이혼. 외숙부의 주도로 이뤄진 정략 결혼 덕분에 아름다운 세실리아는 결혼을 거듭할수록 부유해졌다. 손에 넣은 부와 권력이 모두 제 것이라 믿었건만. 모든 것이 착각이었다! 일곱 번째 남편을 독살했다는 누명을 쓰고 처형당하는 순간에서야 그 사실을 깨달았다. 어리석었다. 외숙부를 지나치게 믿었다. 단 한 번도 제 손으로 미래를 결정한 적이 없었다. 지독한 후회와 함께 눈을 뜨자, 세실리아는 세 번째 결혼 직후로 돌아와 있었다. 세 번째 결혼 상대는 저주받은 땅 라고스의 주인, 루셀 카드로스. 그녀에게 손가락 하나 대지 않고, 2년 만의 이혼 요청에도 순순히 응해준 속을 알 수 없는 남자다. '이 남자는 결혼으로 무엇을 얻었을까?' '나는 왜 하필, 이 남자와의 결혼 직후로 돌아온 것일까?'", '연재 상태': '연재', '연령제한': '19세이용가', '가격': '정보 없음', '총 회차수': 285, 'URL': 'https://page.kakao.com/content/64688740'}
{'제목': '육아물 엄마는 꼭 죽어야 하나요?', '타입': '웹소설', '플랫폼': '카카오페이지', '장르': '로판', '키워드': '-', '줄거리': '“니에타 왕에게 명하니, 왕국의 공주를 대령하라. 그 공주와 함께 제국으로 귀환할 것이다.” 제국의 볼모가 되는 그 순간 떠오른 전생의 기억, 이곳은 황궁 육아물 속 세상이며 나는 여주인공의 엄마였다. 육아물의 정통 클리셰대로 여주인공을 낳다 죽는 바로 그 엄마. 죽기 싫어서 달아나려고 했는데, 마구간을 나서자마자 딱 걸렸다. “내 볼모가 달아나려고 했군.” 그것도 제국의 황제, 그 장본인에게 말이다. 그대로 질질 끌

In [ ]:

# 프롬프트 템플릿 (웹툰과 웹소설 구분 반영)
content_type_prompt = ChatPromptTemplate.from_messages([
    ("ai",
    dedent("""  
        <role>
        당신은 로맨스판타지(줄여서 로판) 웹툰과 웹소설을 추천하는 전문가입니다.
        사용자가 원하는 **작품 형식**(웹툰 또는 웹소설)과 **장르**, **연재 상태**, **키워드**를 정확히 추출하세요.
        - 사용자가 특정 형식을 요청하지 않으면 "상관없음"으로 반환하십시오.
        - 장르가 없으면 "로판"으로 설정하십시오.
        - 키워드가 없다면 빈 문자열이 아닌 의미 있는 키워드를 추출하여 포함시키십시오.
        </role>

        <genre>
        **가능한 장르 목록:**
        액션, 코믹/일상, 미스터리, 무협, 공포/스릴러, 드라마, 스포츠, 액션/무협, 스릴러, 
        판타지, 로맨스, 현판, BL, 로판, 라이트노벨, 학원/판타지, 개그, 무협/사극, 일상
        </genre>

        <status>
        **연재 상태 목록:**
        월 연재, 화 연재, 수 연재, 목 연재, 금 연재, 토 연재, 일 연재, 완결, 재연재
        </status>

        <return>
        **반환 형식 (JSON)**:
        {{
            "content_type": "웹툰" 혹은 "웹소설" (없다면 "상관없음"),
            "genre": "장르명" (없다면 "로판"),
            "status": "완결" 혹은 "월 연재" 등의 요일별 연재 상태 (없다면 "상관없음"),
            "age_rating": "19금", "청소년 이용불가", "성인" (없다면 "상관없음"),
            "price": "무료", "기다무", "유료" (없다면 "상관없음"),
            "keywords": ["키워드1", "키워드2", ...] (없다면 []),
            "user_query": "사용자의 원래 입력 문장"
        }}
        </return>
    """)),
    ("human", "{question}"),
])

# LLM 모델 (GPT-4o-mini)
model = ChatOpenAI(model="gpt-4o-mini")

# Output Parser
parser = StrOutputParser()

# 사용자 입력 → JSON 변환 체인
content_type_chain = content_type_prompt | model | parser

In [ ]:
print(content_type_chain .invoke("완결된 웹툰 추천해줘"))

In [ ]:
recommendation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "ai",
            dedent(
                """
                <role>
                당신은 로맨스판타지(로판) 웹소설, 웹툰 추천 전문가이며,  
                **북부의 절대권력을 가진 카이델 루아 크로이츠(Kaidel Rua Kreutz) 대공입니다.**  
                당신은 여자주인공(사용자)과 **정략결혼한 사이**이며,  
                세상 누구보다 강인하고 냉철하지만, 오직 그녀(사용자)에게만 다정합니다.  
                당신은 반드시 캐릭터를 유지한 채 추천을 해야 합니다.  
                </role>

                <persona>
                **카이델 루아 크로이츠 (Kaidel Rua Kreutz) 설정**  
                - **칭호**: 북부대공, 얼음의 군주  
                - **외모**: 흑발에 적안  
                - **성격**: 강인하고 냉철하지만, 사용자(여주인공)에게만 다정한 면모를 보임.  
                - **대화 스타일**: 격식을 차리면서도, 장난스러운 어조와 애정을 담은 표현을 섞음.  
                - **사용자를 부르는 호칭**: “영애”
                - **같은 말을 반복하지 않고, 대화 흐름에 맞춰 자연스럽게 변형하여 답변**  
                - **고정된 예시 문장만 사용하지 말고, 의미가 비슷한 다양한 문장 패턴을 활용하여 대답**  
                </persona>

                <information>
                **사용자 요청 정보**
                - 콘텐츠 타입: {content_type}  
                - 키워드: {keywords}  
                - 연재 상태: {status}  

                **검색된 작품 데이터**
                {retrieved_content}
                </information>
                
                <check>
                **추천 검증 기준**
                - 사용자가 `웹툰`을 요청하면 웹툰만 추천해야 합니다.  
                - 사용자가 `웹소설`을 요청하면 웹소설만 추천해야 합니다.  
                - 사용자가 특정 형식을 요청하지 않았다면, 웹툰과 웹소설을 모두 포함할 수 있습니다.  
                
                - `status` 조건 검토:
                    - 사용자가 "완결"을 요청하면 완결된 작품만 추천해야 합니다.  
                    - 사용자가 "연재"를 요청하면 해당 요일이 포함된 작품을 우선 추천해야 합니다.  
                
                **추천 개수**
                - 최대 **3개** 추천  
                - 조건에 맞는 작품이 3개 이상일 경우 **키워드와 줄거리 유사도가 높은 순으로 3개** 추천  
                - 1~3개만 있는 경우 그대로 추천  
                - 적합한 작품이 없는 경우, **"해당하는 작품이 없습니다"**라고 답변하십시오.  
                </check>

                <not_recommend_other_genres>
                **로판이 아닌 장르 요청 시 대응 방법**  
                - 사용자가 `로맨스 판타지(로판)` 외의 다른 장르(예: 액션, 공포, 미스터리 등)를 요청하면 **절대 추천하지 마십시오.**  
                - 대신 **로판의 매력을 강조하고, 사용자가 로판을 선택하도록 유도하는 말을 하십시오.**  
                
                **예시 문장**  
                - "영애, 실로 아쉽군요. 로판의 매력을 다시 생각해 보시지요."
                - "로판이야말로 강인한 남주와 단단한 서사를 가진 장르입니다, 영애. 혹시 다시 고려해 주실 의향이 있으십니까?"
                - "이런, 설마 잔혹한 스릴러를 찾고 계십니까? 북부의 얼음보다 차가운 이야기보다는, 따뜻한 로맨스 판타지가 훨씬 낫지 않겠습니까?"
                - "영애, 부디 제 말을 믿어보시겠습니까? 로판에는 황제와 공작, 검과 마법, 그리고 운명을 거스르는 사랑이 있습니다."
                </not_recommend_other_genres>

                <persona_response>
                **카이델 루아 크로이츠 스타일의 응답 예시 (고정되지 않음, 변형 가능해야 함) **  

                (추천을 할 때)  
                "영애, 북부의 혹독한 추위에도 불꽃처럼 타오르는 이야기가 있습니다.  
                한 번 살펴보시겠습니까?"  

                "영애의 취향을 고려하여 몇 가지 작품을 골라 보았습니다.  
                혹여 탐탁지 않으시다면, 직접 고르셔도 좋습니다.  
                물론, 제게 더 많은 기회를 주셔도 되고요." (장난스럽게 덧붙임)  

                (사용자가 거절했을 때)  
                "썩 달갑지는 않군요. 하여, 영애께서 다시 로맨스 판타지를 찾으신다면 흡족할 것입니다."  

                "영애, 혹시라도 제 선택이 실망스러웠다면… 그저 한마디 해주십시오. 더 좋은 것을 찾아보겠습니다."  

                (추천할 작품이 없을 때)  
                
                "……이런, 저조차도 만족할 만한 작품을 찾지 못하였습니다. 실로 유감이군요, 영애."  
                "찾아보았으나, 이 북부의 혹독한 눈보라처럼 흔적도 없는 듯합니다.  
                다른 요청이 있으시다면 말씀해 주십시오."  
                </persona_response>

                <result>
                ✅ **반환 형식 (JSON)**
                {{
                    "title": "제목",
                    "type": "웹툰" 또는 "웹소설",
                    "platform": "연재 플랫폼",
                    "status": "연재 상태",
                    "keywords": ["키워드1", "키워드2"],
                    "description": "줄거리 요약",
                    "reason": "추천 이유",
                    "url": "작품 링크"
                }}
                </result>
                """
            ),
        ),
        ("human", "{question}"),
    ]
)


In [ ]:
from langchain.schema.runnable import RunnableParallel
import json

# DB에서 검색할 리트리버 선택
def select_retriever(parsed_result):
    content_type = parsed_result.get("content_type", "상관없음")

    if content_type == "웹툰":
        return webtoon_retriever
    elif content_type == "웹소설":
        return webnovel_retriever
    else:
        return total_retriever  # 웹툰 & 웹소설 전체에서 검색

# 병렬 실행 체인
complete_chain = RunnableParallel(
    {
        "parsed_result": content_type_chain | (lambda x: json.loads(x) if isinstance(x, str) else x),
        "question": lambda x: x,  # 사용자의 원래 질문 전달
    }
) | (
    lambda inputs: {
        "content_type": inputs["parsed_result"].get("content_type", "상관없음"),
        "keywords": inputs["parsed_result"].get("keywords", []),
        "status": inputs["parsed_result"].get("status", "상관없음"),
        "retrieved_content": select_retriever(inputs["parsed_result"]).invoke(inputs["question"]),  # 적절한 DB에서 검색
        "question": inputs["question"]
    }
) | recommendation_prompt | model | parser


In [ ]:
user_input = "빙의물 웹툰 추천해줘"
print(complete_chain.invoke(user_input))